# <center>JSONiq Update Facility sandbox</center>


The following cell is meant for Google Colab, which does not have the right Java version.

In [ ]:
# We make sure Java 17 is installed.
!apt update
!apt install openjdk-17-jdk
!java -version

In [60]:
!pip install jsoniq


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


We now initialize a RumbleDB session and test that it works.

In [91]:
from jsoniq import RumbleSession

rumble = RumbleSession.builder.withDelta().getOrCreate();

print(rumble.jsoniq('{ "foo": [ 6*7 ] }').json());

({'foo': [42]},)


For convenience, we also activate the JSONiq magic.

In [92]:
%load_ext jsoniqmagic
# We will display the first 5 items. This parameter can be changed.
rumble.getRumbleConf().setResultSizeCap(20)

The jsoniqmagic extension is already loaded. To reload it, use:
  %reload_ext jsoniqmagic


JavaObject id=o514

We create a new collection

In [96]:
%%jsoniq -u -t
create collection table("foo") with for $i in 1 to 3 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.
Response time: 0.4432852268218994 ms


In [97]:
%%jsoniq -df -t
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
|  3|[Alice, Bob, Char...|      2|       2.0|                0|        |            foo|
+---+--------------------+-------+----------+-----------------+--------+---------------+

Response time: 0.1620190143585205 ms


We insert new data -- remember the snapshot semantics, all updates are computed against the original table, and then applied jointly.

In [98]:
%%jsoniq -u -t
insert for $i in 4 to 6 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection table("foo"),
insert for $i in 7 to 10 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection table("foo"),
insert for $i in 11 to 13 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before table("foo")[position() eq 2] into collection,
insert for $i in 14 to 16 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after table("foo")[position() eq 2] into collection


Updates applied successfully.
Response time: 2.8711822032928467 ms


In [99]:
%%jsoniq -df -t
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |            foo|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |            foo|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |            foo|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |            foo|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |            foo|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
| 14|[Alice, Bob, Cha

We delete a record

In [100]:
%%jsoniq -u -t
delete table("foo")[$$.foo eq 15] from collection

Updates applied successfully.
Response time: 0.6823310852050781 ms


In [101]:
%%jsoniq -df -t
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |            foo|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |            foo|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |            foo|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |            foo|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |            foo|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
| 14|[Alice, Bob, Cha

We edit a record

In [102]:
%%jsoniq -u -t
edit table("foo")[$$.foo eq 14] into { "foo" : 17, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 0.9289381504058838 ms


In [103]:
%%jsoniq -df -t
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |            foo|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |            foo|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |            foo|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |            foo|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |            foo|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
| 17|[Alice, Bob, Dan

In [104]:
%%jsoniq -t
rename json table("foo")[1].foo as "bar2";

Response time: 1.0600237846374512 ms


In [105]:
%%jsoniq -df -t
table("foo")

+----+--------------------+-------+----------+----+-----------------+--------+---------------+
| foo|                 bar|__rowID|__rowOrder|bar2|__mutabilityLevel|__pathIn|__tableLocation|
+----+--------------------+-------+----------+----+-----------------+--------+---------------+
|NULL|[Alice, Bob, Char...|      9|  -75000.0|   4|                0|        |            foo|
|   5|[Alice, Bob, Char...|     10|  -50000.0|NULL|                0|        |            foo|
|   6|[Alice, Bob, Char...|     11|  -25000.0|NULL|                0|        |            foo|
|   1|[Alice, Bob, Char...|      0|       0.0|NULL|                0|        |            foo|
|  11|[Alice, Bob, Char...|      3|      0.25|NULL|                0|        |            foo|
|  12|[Alice, Bob, Char...|      4|       0.5|NULL|                0|        |            foo|
|  13|[Alice, Bob, Char...|      5|      0.75|NULL|                0|        |            foo|
|   2|[Alice, Bob, Char...|      1|       1.0|NULL

In [106]:
%%jsoniq -u -t
delete collection table("foo")

Updates applied successfully.
Response time: 0.020617961883544922 ms


In [107]:
%%jsoniq -u -t
create collection delta-file("foobar") with for $i in 1 to 3 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.
Response time: 0.34624385833740234 ms


In [108]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |delta.`file:/User...|
|  3|[Alice, Bob, Char...|      2|       2.0|                0|        |delta.`file:/User...|
+---+--------------------+-------+----------+-----------------+--------+--------------------+

Response time: 0.18596506118774414 ms


In [109]:
%%jsoniq -u -t
insert for $i in 4 to 6 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection delta-file("foobar"),
insert for $i in 7 to 10 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection delta-file("foobar"),
insert for $i in 11 to 13 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before delta-file("foobar")[position() eq 2] into collection,
insert for $i in 14 to 16 return { "foobar" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after delta-file("foobar")[position() eq 2] into collection


Updates applied successfully.
Response time: 3.0012519359588623 ms


In [110]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |delta.`file:/User...|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |delta.`file:/User...|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |delta.`file:/User...|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |delta.`file:/User...|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |delta.`file:/User...|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|               

In [111]:
%%jsoniq -u -t
delete delta-file("foobar")[$$.foo eq 15] from collection

Updates applied successfully.
Response time: 0.6445589065551758 ms


In [112]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |delta.`file:/User...|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |delta.`file:/User...|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |delta.`file:/User...|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |delta.`file:/User...|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |delta.`file:/User...|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|               

In [113]:
%%jsoniq -u -t
edit delta-file("foobar")[$$.foo eq 14] into { "foo" : 17, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 0.9985339641571045 ms


In [114]:
%%jsoniq -df -t
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |delta.`file:/User...|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |delta.`file:/User...|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |delta.`file:/User...|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |delta.`file:/User...|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |delta.`file:/User...|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|               

In [115]:
%%jsoniq -u -t
delete collection delta-file("foobar")

Updates applied successfully.
Response time: 0.020194053649902344 ms
